In [ ]:
import sqlite3
import sys

# Database functions
def create_connection():
    connection = sqlite3.connect('finance.db')
    return connection

def create_tables():
    connection = create_connection()
    cursor = connection.cursor()
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS transactions (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        type TEXT NOT NULL,
                        category TEXT NOT NULL,
                        amount REAL NOT NULL,
                        date TEXT NOT NULL
                      )''')
    
    cursor.execute('''CREATE TABLE IF NOT EXISTS budgets (
                        id INTEGER PRIMARY KEY AUTOINCREMENT,
                        category TEXT NOT NULL,
                        amount REAL NOT NULL,
                        month TEXT NOT NULL
                      )''')
    
    connection.commit()
    connection.close()

# Finance Manager class
class FinanceManager:
    def __init__(self):
        self.connection = create_connection()
        
    def add_transaction(self, trans_type, category, amount, date):
        cursor = self.connection.cursor()
        cursor.execute('INSERT INTO transactions (type, category, amount, date) VALUES (?, ?, ?, ?)', 
                       (trans_type, category, amount, date))
        self.connection.commit()
        
    def set_budget(self, category, amount, month):
        cursor = self.connection.cursor()
        cursor.execute('INSERT INTO budgets (category, amount, month) VALUES (?, ?, ?)', 
                       (category, amount, month))
        self.connection.commit()
        
    def view_transactions(self):
        cursor = self.connection.cursor()
        cursor.execute('SELECT * FROM transactions')
        rows = cursor.fetchall()
        for row in rows:
            print(row)
            
    def generate_report(self, month):
        cursor = self.connection.cursor()
        cursor.execute('SELECT SUM(amount) FROM transactions WHERE type = "income" AND strftime("%Y-%m", date) = ?', (month,))
        total_income = cursor.fetchone()[0] or 0
        
        cursor.execute('SELECT SUM(amount) FROM transactions WHERE type = "expense" AND strftime("%Y-%m", date) = ?', (month,))
        total_expenses = cursor.fetchone()[0] or 0
        
        cursor.execute('SELECT category, amount FROM budgets WHERE month = ?', (month,))
        budgets = cursor.fetchall()
        
        print(f"Financial Report for {month}")
        print(f"Total Income: ${total_income}")
        print(f"Total Expenses: ${total_expenses}")
        print(f"Remaining Budget: ${total_income - total_expenses}")
        print("\nBudgets:")
        for budget in budgets:
            print(f"{budget[0]}: ${budget[1]}")
            
    def close_connection(self):
        self.connection.close()

# Main application logic
def main():
    create_tables()
    manager = FinanceManager()
    
    while True:
        print("\nPersonal Finance Manager")
        print("1. Add Income")
        print("2. Add Expense")
        print("3. Set Budget")
        print("4. View Transactions")
        print("5. Generate Report")
        print("6. Exit")
        
        choice = input("Enter your choice: ")
        
        if choice == '1':
            category = input("Enter category: ")
            amount = float(input("Enter amount: "))
            date = input("Enter date (YYYY-MM-DD): ")
            manager.add_transaction('income', category, amount, date)
            print("Income added successfully.")
            
        elif choice == '2':
            category = input("Enter category: ")
            amount = float(input("Enter amount: "))
            date = input("Enter date (YYYY-MM-DD): ")
            manager.add_transaction('expense', category, amount, date)
            print("Expense added successfully.")
            
        elif choice == '3':
            category = input("Enter category: ")
            amount = float(input("Enter amount: "))
            month = input("Enter month (YYYY-MM): ")
            manager.set_budget(category, amount, month)
            print("Budget set successfully.")
            
        elif choice == '4':
            manager.view_transactions()
            
        elif choice == '5':
            month = input("Enter month (YYYY-MM): ")
            manager.generate_report(month)
            
        elif choice == '6':
            manager.close_connection()
            print("Goodbye!")
            break
            
        else:
            print("Invalid choice. Please try again.")

if __name__ == "__main__":
    main()
